## Setup the ssh connection to the remote server

In [71]:
import paramiko
from getpass import getpass
import os


# Check to see if result text files exists
rootdir = 'remote_files'
fList = []
# Crawl through the provided directory
for root, subfolders, filenames in os.walk(rootdir):
    for f in filenames:
        fileList = root + "/" + f
        fList.append(fileList)

# Check to see if ssh_results file exists
for eachFile in fList:
    if os.path.exists(eachFile):
        # If so ask user to delete file
        user_input = input("The file: "+ eachFile + " exists do you want to delete the file?(Y/N)")

        # If yes delete the file
        if user_input == "Y":
            os.remove(eachFile)


# create the password prompt
thePass = getpass(prompt="Please enter your SSH password")

# Host information
host = "192.168.6.71"
port = 2222
username = "emily.crawford"
password = thePass

# try connecting via ssh using paramiko
try :
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(host,port,username,password)
# if fails from authentication error print that it failed due to authentication failure
except paramiko.AuthenticationException:
    print ("Authentication failed")

## This is a function for running a list of commands and then saving them to a designated output file If the sudo command is used then the password will need to be provided.


In [ ]:
def runListOfCommands(commandsLists,password,outputFile):

    for eachCMD in commandsLists:

        # Get the output from the command
        stdin, stdout, stderr = ssh.exec_command(eachCMD)

        # input password for running sudo command
        stdin.write(password + "\n")

        # Get results from stout
        lines = stdout.readlines()

        # Convert the list to a string
        output = ''.join(lines)

        # Header output
        sepHeader = '' + '### BEGIN ' + eachCMD + ' ###\n\n'

        # Footer
        sepFooter = '' + '### END ' + eachCMD  + ' ###\n\n'

        # Concatenates the header, command, footer
        cmd_output = sepHeader + output + sepFooter

        # save the cmd_output to a file
        with open(outputFile, 'a') as f:
            f.write(cmd_output)
        # print the results out to the console as well
        print(cmd_output)

## This function is for downloading remote files to the local system

In [ ]:
def downloadFiles (files,download_dir):
    # Loop through the files and download them
    for eachFile in files:

        # Create a list from the filename
        x = eachFile.split("/")

        # -1 is used to pluck the last element in a list.
        filename = x[-1]

        # Local path to store files
        local_path = str(download_dir) +  "/" +str(filename)

        # get the file
        sftp.get(eachFile,local_path)


## For task 2: upload the Kraken binary and run it against /usr/bin/, /usr/sbin, /usr/local/bin, and /usr/local/sbin

In [73]:
# start SFTP session
sftp = ssh.open_sftp()

# the local file on the system to upload
localFile = "kraken"
# the remote path to upload the file to
remotePath = "/home/emily.crawford/kraken"

# push the file to the linux server in the correct path
sftp.put(localFile, remotePath)

# Run a list of commands, these will give the kraken binary executable permissions
command = ['sudo -S chmod +x kraken','sudo -S /home/emily.crawford/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin']

# file to save the output in
output = "results/kraken-results.txt"

# Run all the commands, and save the result to the directory results as well as printing them out
runListOfCommands(command,thePass,output)

### BEGIN dir ###

auth.log  kraken  ps-aux-results.txt
### END dir ###


Errors found
[sudo] password for emily.crawford: 
### BEGIN sudo -S chmod +x kraken ###

### END sudo -S chmod +x kraken ###


Errors found
[sudo] password for emily.crawford: 
### BEGIN sudo -S /home/emily.crawford/kraken --folder /usr/bin --folder  --folder /usr/sbin/   --folder /usr/local/bin  --folder /sbin  --folder /usr/local/sbin  --folder /bin ###

INFO[0000] Looking for configuration file with name config.yaml 
INFO[0000] No configuration file found, generating a default one... 
INFO[0000] Loading Yara rules...                        
INFO[0000] Scanning running processes...                
WARN[0004] DETECTION! Malicious process detected as foundGoBinary  pid=3138 process=ls
WARN[0005] DETECTION! Malicious process detected as foundGoBinary  pid=25680 process=snapd
INFO[0018] Scanning autoruns...                         
WARN[0019] DETECTION! Malicious autorun detected as foundGoBinary  image_path=/usr/b

## Task 3 look for any malicious process IDs that were running as found by kraken

In [51]:
# suspicious pid to investigate
pid = 3138

# command to run
command = "sudo -S lsof -p" + str(pid)

# file to save the output in
output = "results/lsof-pid-results.txt"

runListOfCommands(command,thePass,output)

### BEGIN sudo -S lsof -p3138 ###

COMMAND  PID     USER   FD      TYPE DEVICE SIZE/OFF   NODE NAME
ls      3138 p.dalton  cwd       DIR    8,1     4096 258130 /home/p.dalton/test-lk
ls      3138 p.dalton  rtd       DIR    8,1     4096      2 /
ls      3138 p.dalton  txt       REG    8,1  6597697 258133 /usr/local/bin/ls
ls      3138 p.dalton  DEL       REG    8,1            3453 /usr/lib/x86_64-linux-gnu/libc-2.31.so
ls      3138 p.dalton  DEL       REG    8,1            3466 /usr/lib/x86_64-linux-gnu/libpthread-2.31.so
ls      3138 p.dalton  DEL       REG    8,1            3449 /usr/lib/x86_64-linux-gnu/ld-2.31.so
ls      3138 p.dalton    0r      CHR    1,3      0t0      6 /dev/null
ls      3138 p.dalton    1w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log
ls      3138 p.dalton    2w      REG    8,1       85 258135 /home/p.dalton/test-lk/.d.log
ls      3138 p.dalton    3r      CHR    1,3      0t0      6 /dev/null
ls      3138 p.dalton    4u      REG    8,1        4 258

## Task 4: Download any suspicious files, binaries, or log files

In [52]:
# start SFTP session
sftp = ssh.open_sftp()

# List of files to download
files = ['/home/p.dalton/test-lk/.d.log', '/home/p.dalton/test-lk/d.pid']

# directory to download the remote files to
downloadDir = "remote_files"

# download the files
downloadFiles(files,downloadDir)

## Task 5: Check running processes, /etc/passwd, and /var/log/auth.log

In [68]:

# Check running processes
command = ['sudo -S ps aux > /home/emily.crawford/ps-aux-results.txt','sudo -S cp /var/log/auth.log .', 'sudo -S chmod +r auth.log']

output = "results/check_processes_and_etc_passwd_and_var_log_auth.txt"

# run the commands
runListOfCommands(command,thePass,output)

# start SFTP session
sftp = ssh.open_sftp()

# List of files to download
files = ['/etc/passwd', 'auth.log','ps-aux-results.txt']

# the directory to download the files to
downloadDir = "remote_files"

# Download the files now
downloadDir(files,downloadDir)


Errors found
[sudo] password for emily.crawford: 
### BEGIN sudo -S ps aux > /home/emily.crawford/ps-aux-results.txt ###

### END sudo -S ps aux > /home/emily.crawford/ps-aux-results.txt ###


Errors found
[sudo] password for emily.crawford: 
### BEGIN sudo -S cp /var/log/auth.log . ###

### END sudo -S cp /var/log/auth.log . ###


Errors found
[sudo] password for emily.crawford: 
### BEGIN sudo -S chmod +r auth.log ###

### END sudo -S chmod +r auth.log ###




In [69]:
# close the ssh session
ssh.close()

## Task 5: Questions

**What is the process name?**



**Which user is running the process?**


**Has the user logged in before via SSH?**
    User `p.dalton` logged in with ssh before.

**When did the user login?**
    The user logged in first on Feb 27 at 04:44:14

**How long was the login session?**
    The user was logged in for 10 seconds, was disconnected, before logging back in again.

**What is located in the unauthorized user’s home directory?**


Download any suspicious files.